# Backfill Demo: Setup Notebook

This notebook sets up all required tables for demonstrating Databricks backfilling capabilities.

## Tables Created:
1. **Calendar Table** - Business day calendar (2020-2026)
2. **Source Table** - Historical data with business dates
3. **Destination Table** - Partitioned target table for processed data
4. **Backfill Log Table** - Tracks backfill operations and their status

In [0]:
# Load Configuration
import sys

# Get current notebook path dynamically
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
workspace_path = f"/Workspace{notebook_path}"
base_path = workspace_path.rsplit('/', 1)[0]

# Add to sys.path for importing config
sys.path.append(base_path)

from config import CATALOG, SCHEMA, SOURCE_TABLE, DEST_TABLE, CALENDAR_TABLE, BACKFILL_LOG_TABLE, print_config

# Display configuration
print_config()

📁 Catalog: demos
📂 Schema: backfill_demo
📊 Tables:
  • Source: demos.backfill_demo.source_data
  • Destination: demos.backfill_demo.destination_data
  • Calendar: demos.backfill_demo.calendar
  • Backfill Log: demos.backfill_demo.backfill_log


In [0]:
# Create Catalog and Schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
print(f"✓ Catalog '{CATALOG}' ready")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
print(f"✓ Schema '{CATALOG}.{SCHEMA}' ready")

✓ Catalog 'demos' ready
✓ Schema 'demos.backfill_demo' ready


In [0]:
# Cell 3: Create DDLs - Managed Tables

# 3.1 Calendar Table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CALENDAR_TABLE} (
        calendar_date DATE NOT NULL,
        calendar_year INT,
        quarter_of_the_year INT,
        month_of_the_year INT,
        week_of_the_month INT,
        us_business_or_holiday_flag STRING,
        global_business_or_holiday_flag STRING
    )
    USING DELTA
""")
print(f"✓ Created table: {CALENDAR_TABLE}")

# 3.2 Source Data Table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {SOURCE_TABLE} (
        position_date DATE,
        id BIGINT,
        value DOUBLE,
        category STRING
    )
    USING DELTA
""")
print(f"✓ Created table: {SOURCE_TABLE}")

# 3.3 Destination Data Table (Partitioned)
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {DEST_TABLE} (
        position_date DATE,
        id BIGINT,
        value DOUBLE,
        category STRING
    )
    USING DELTA
    PARTITIONED BY (position_date)
""")
print(f"✓ Created table: {DEST_TABLE}")

# 3.4 Backfill Log Table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {BACKFILL_LOG_TABLE} (
        run_id STRING NOT NULL,
        position_date DATE,
        job_name STRING,
        job_id STRING,
        status STRING,
        start_time TIMESTAMP,
        end_time TIMESTAMP,
        backfill_job_id STRING,
        error_message STRING,
        creator_user_name STRING,
        run_page_url STRING,
        retry_metadata STRING COMMENT 'JSON: {is_retry, original_run_id, retry_count, retry_triggered_by}'
    )
    USING DELTA
""")
print(f"✓ Created table: {BACKFILL_LOG_TABLE}")

✓ Created table: demos.backfill_demo.calendar
✓ Created table: demos.backfill_demo.source_data
✓ Created table: demos.backfill_demo.destination_data
✓ Created table: demos.backfill_demo.backfill_log


## Data Generation

We'll populate the tables with sample data using efficient Spark operations (no Python loops!).

In [0]:
# Populate Source Table with Sample Data
import numpy as np
from pyspark.sql.types import StructType, StructField, DateType, LongType, DoubleType, StringType
from pyspark.sql import functions as F
from datetime import datetime

# Set seed for reproducibility
np.random.seed(42)

# Generate business dates for 2024-2025
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 11, 20)

# Use Spark SQL to generate dates efficiently
dates_df = spark.sql(f"""
    SELECT explode(sequence(to_date('{start_date.date()}'), to_date('{end_date.date()}'), interval 1 day)) as position_date
""").filter(F.dayofweek(F.col("position_date")).between(2, 6))  # Monday=2 to Friday=6

# Generate 10 records per date with cross join
records_df = spark.range(0, 10).selectExpr("id as id")

# Create sample data using cross join
sample_data = dates_df.crossJoin(records_df).select(
    F.col("position_date"),
    F.col("id"),
    (F.rand(seed=42) * 100).alias("value"),
    F.array(F.lit("A"), F.lit("B"), F.lit("C"))[(F.rand(seed=43) * 3).cast("int")].alias("category")
)

# Write to source table
sample_data.write.format("delta").mode("overwrite").saveAsTable(SOURCE_TABLE)

row_count = sample_data.count()
date_count = dates_df.count()
print(f"✓ Populated {SOURCE_TABLE}")
print(f"  └─ {row_count:,} rows across {date_count} business days")

✓ Populated demos.backfill_demo.source_data
  └─ 4,940 rows across 494 business days


In [0]:
# Populate Calendar Table (2020-2026) with Real US Holidays
from pyspark.sql import functions as F
from datetime import datetime
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar

# Generate date range from 2020 to 2026
start_date = datetime(2020, 1, 1)
end_date = datetime(2026, 12, 31)

# Get US federal holidays using pandas
cal = USFederalHolidayCalendar()
us_holidays = cal.holidays(start=start_date, end=end_date)
us_holiday_dates = set(us_holidays.date)
print(f"📅 Loaded {len(us_holiday_dates)} US federal holidays (2020-2026)")

# Generate all dates using Spark SQL
all_dates_df = spark.sql(f"""
    SELECT 
        explode(sequence(to_date('{start_date.date()}'), to_date('{end_date.date()}'), interval 1 day)) as calendar_date
""")

# Convert to pandas for holiday checking, then back to Spark
dates_pd = all_dates_df.toPandas()

# Ensure calendar_date is datetime type
dates_pd['calendar_date'] = pd.to_datetime(dates_pd['calendar_date'])

# Add calendar columns
dates_pd['calendar_year'] = dates_pd['calendar_date'].dt.year
dates_pd['quarter_of_the_year'] = dates_pd['calendar_date'].dt.quarter
dates_pd['month_of_the_year'] = dates_pd['calendar_date'].dt.month
dates_pd['week_of_the_month'] = ((dates_pd['calendar_date'].dt.day - 1) // 7) + 1

# Determine business day vs holiday
# B = Business day (Monday-Friday, not a US federal holiday)
# H = Holiday (Weekend or US federal holiday)
def is_business_day(dt):
    # Weekend check
    if dt.weekday() >= 5:  # Saturday=5, Sunday=6
        return 'H'
    # US holiday check
    if dt.date() in us_holiday_dates:
        return 'H'
    return 'B'

dates_pd['us_business_or_holiday_flag'] = dates_pd['calendar_date'].apply(is_business_day)
dates_pd['global_business_or_holiday_flag'] = dates_pd['us_business_or_holiday_flag']  # Same for this demo

# Convert calendar_date to date (not datetime) for Pandas
dates_pd['calendar_date'] = dates_pd['calendar_date'].dt.date

# Convert back to Spark DataFrame with explicit date type
calendar_df = spark.createDataFrame(dates_pd)

# Write to calendar table with overwriteSchema to handle any schema changes
calendar_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(CALENDAR_TABLE)

# Get statistics
stats = calendar_df.groupBy("us_business_or_holiday_flag").count().collect()
total_days = sum([row['count'] for row in stats])
business_days = [row['count'] for row in stats if row['us_business_or_holiday_flag'] == 'B'][0]
holidays_count = total_days - business_days

print(f"✓ Populated {CALENDAR_TABLE}")
print(f"  └─ Total days: {total_days:,} (2020-2026)")
print(f"  └─ Business days: {business_days:,}")
print(f"  └─ Holidays/Weekends: {holidays_count:,}")
print(f"\n📋 Sample US Federal Holidays Marked:")
for dt in sorted(list(us_holiday_dates))[:5]:
    print(f"  • {dt}")

📅 Loaded 76 US federal holidays (2020-2026)
✓ Populated demos.backfill_demo.calendar
  └─ Total days: 2,557 (2020-2026)
  └─ Business days: 1,751
  └─ Holidays/Weekends: 806

📋 Sample US Federal Holidays Marked:
  • 2020-01-01
  • 2020-01-20
  • 2020-02-17
  • 2020-05-25
  • 2020-07-03


In [0]:
# Verify Setup - Quick Data Preview
print("\n📊 Quick Data Preview:\n")

# Show sample from each table
print("Calendar Table:")
display(spark.sql(f"SELECT * FROM {CALENDAR_TABLE} WHERE calendar_date = '2025-01-15' LIMIT 5"))

print("Source Data Table:")
display(spark.sql(f"SELECT * FROM {SOURCE_TABLE} WHERE position_date = '2025-01-15' LIMIT 5"))

print("Table Statistics:")
print(f"  • Calendar: {spark.table(CALENDAR_TABLE).count():,} rows")
print(f"  • Source Data: {spark.table(SOURCE_TABLE).count():,} rows")
print(f"  • Destination: {spark.table(DEST_TABLE).count():,} rows (empty)")
print(f"  • Backfill Log: {spark.table(BACKFILL_LOG_TABLE).count():,} rows (empty)")


📊 Quick Data Preview:

Calendar Table:


calendar_date,calendar_year,quarter_of_the_year,month_of_the_year,week_of_the_month,us_business_or_holiday_flag,global_business_or_holiday_flag
2025-01-15,2025,1,1,3,B,B


Source Data Table:


position_date,id,value,category
2025-01-15,3,73.44850394468924,B
2025-01-15,4,13.795336317998563,A
2025-01-15,8,49.539320824317414,B
2025-01-15,9,60.99716667551187,B
2025-01-15,7,15.084687792490248,B


Table Statistics:
  • Calendar: 2,557 rows
  • Source Data: 4,940 rows
  • Destination: 0 rows (empty)
  • Backfill Log: 0 rows (empty)


## Setup Complete! 🎉

All tables have been created and populated successfully.

### What's Ready:
- ✅ **Calendar Table** - Business day calendar (2020-2026)
- ✅ **Source Table** - Sample data with business dates
- ✅ **Destination Table** - Partitioned target (empty, ready for backfill)
- ✅ **Backfill Log Table** - Operation tracking (empty, ready for use)

### Next Steps:
1. **`02_process_data.ipynb`** - Create the data processing notebook
2. **`03_backfill_orchestrator.ipynb`** - Orchestrate and monitor backfill operations

---
*Note: This setup uses managed Delta tables for simplicity. All data is stored in your catalog's default location.*